In [22]:
from sqlglot import parse_one, exp
from sqlglot.dialects.tsql import TSQL
import pypyodbc as odbc
odbc.lowercase = False
import configparser
import copy
from collections import defaultdict
from collections import OrderedDict 
import pandas as pd
import pypyodbc as odbc
import configparser
import os
import json
import re
from parse_nodes import *

odbc.lowercase = False

In [23]:
queries = [] # queries content list
files = [] # file names list
for file in os.listdir("gpt-sql-queries-extracted3"):
    files.append(file.split('.')[0])
    with open(f'gpt-sql-queries-extracted3/{file}', 'r') as file:
        data = json.load(file)

    queries.append(data)

print(files[0])
queries[0]


json_data0


{'modified_SQL_query': 'CREATE VIEW CUSTOMER_ORDER AS SELECT c.customer_id, c.first_name, c.last_name, c.email, recent_orders.order_id, recent_orders.order_date, recent_orders.total_order_amount, recent_payments.total_payment_amount, recent_order_items.product_name, recent_order_items.category_name, recent_order_items.quantity, recent_order_items.unit_price, recent_order_items.total_item_amount, CAST(avg_product_rating.avg_rating AS DECIMAL(3,2)) AS avg_product_rating, CAST(total_customer_spend.total_spent AS DECIMAL(10,2)) AS total_customer_spent FROM customers c JOIN subquery1_1 recent_orders ON c.customer_id = recent_orders.customer_id JOIN subquery1_2 recent_payments ON recent_orders.order_id = recent_payments.order_id JOIN subquery1_3 recent_order_items ON recent_orders.order_id = recent_order_items.order_id JOIN subquery1_4 avg_product_rating ON recent_order_items.product_id = avg_product_rating.product_id JOIN subquery1_5 total_customer_spend ON c.customer_id = total_customer_sp

In [24]:
query_subqueries = dict(reversed(list(queries[0]['subquery_dictionary'].items())))
query_subqueries['main_query'] = queries[0]['modified_SQL_query']
query_subqueries


{'subquery1_5': "SELECT o.customer_id, SUM(o.total_amount) AS total_spent FROM orders o WHERE o.status = 'Completed' GROUP BY o.customer_id",
 'subquery1_4': 'SELECT r.product_id, AVG(r.rating) AS avg_rating FROM reviews r GROUP BY r.product_id',
 'subquery1_3': 'SELECT oi.order_id, p.product_name, cat.category_name, oi.quantity, oi.unit_price, (oi.quantity * oi.unit_price) AS total_item_amount FROM order_items oi JOIN products p ON oi.product_id = p.product_id JOIN categories cat ON p.category_id = cat.category_id',
 'subquery1_2': 'SELECT p.order_id, SUM(p.payment_amount) AS total_payment_amount FROM payments p GROUP BY p.order_id',
 'subquery1_1': "SELECT o.order_id, o.customer_id, o.order_date, o.total_amount AS total_order_amount FROM orders o WHERE o.order_date >= DATEADD(month, -1, GETDATE()) AND o.status = 'Completed'",
 'main_query': 'CREATE VIEW CUSTOMER_ORDER AS SELECT c.customer_id, c.first_name, c.last_name, c.email, recent_orders.order_id, recent_orders.order_date, recent

In [25]:
def on_statement(select_statement):
    """Function to extract the on condition from the join statements, (on column = column)"""

    # from expression
    joins = list(select_statement.find_all(exp.Join))
    on_conditions = []
    for join in joins:
        try:
        
            on_conditions.append(f"{list(join.find_all(exp.EQ))[0].this.table}.{list(join.find_all(exp.EQ))[0].this.this} = {list(join.find_all(exp.EQ))[0].expression.table}.{list(join.find_all(exp.EQ))[0].expression.this}")
        except:
            return []
    if joins != []:
        return on_conditions
    else:
        return []


  

 #[EQ(
 #this=Column(
 #  this=Identifier(this=customer_id, quoted=False),
 #  table=Identifier(this=customers, quoted=False)),
 #expression=Column(
 #  this=Identifier(this=customer_id, quoted=False),
 #  table=Identifier(this=subquery1_1, quoted=False)))]

In [26]:

nodes = [] # list of dictionaries with the nodes
filename = 'query1'
for name_query in query_subqueries:
    print(name_query)

    #query = query_subqueries[name_query].strip("()") # strip paranthesis from subqueries

    query = query_subqueries[name_query]
    if query.startswith("("):
        query = query.strip("()")
    else:
        pass
    
    ast = parse_query(query) # get parsed tree

    
    tree = replace_aliases(query) # get transformed tree without table aliases
    
    #print(get_tables(tree))
    # parse selects before getting statements
    select_statement_big = tree.find_all(exp.Select)

    source_tables = []
    for select in list(select_statement_big):
        
        source_table, where_exp = get_statements(select) 

        on_condition = on_statement(select)

        source_tables += source_table

    if 'subquery' in name_query: # if the query is a subquery then the name is the dict key, else the name is the target table

        target_node = name_query
        nodes.append({'NAME_NODE': target_node, 'LABEL_NODE': f'{filename}@{target_node}', 'WHERE': where_exp, 'FUNCTION': 'subquery', 'ON': on_condition})
    else:
        try:
            target_node = list(ast.find_all(exp.Create))[0].this.this.this
            nodes.append({'NAME_NODE': target_node,'LABEL_NODE': target_node, 'WHERE': where_exp, 'FUNCTION': 'target', 'ON': on_condition})
        except IndexError:
            target_node = list(ast.find_all(exp.Insert))[0].this.this
            nodes.append({'NAME_NODE': target_node,'LABEL_NODE': target_node, 'WHERE': where_exp, 'FUNCTION': 'target', 'ON': on_condition})


    for table in source_tables:
        if table not in [node['NAME_NODE'] for node in nodes]:
            if 'subquery' in table: 
                nodes.append({'NAME_NODE': table,'LABEL_NODE': f'{filename}@{table}', 'WHERE': None, 'FUNCTION': 'subquery', 'ON': on_condition})
            else:
                nodes.append({'NAME_NODE': table,'LABEL_NODE': table, 'WHERE': None, 'FUNCTION': 'table', 'ON': ''})


nodes = pd.DataFrame(nodes)
nodes



subquery1_5
subquery1_4
subquery1_3
subquery1_2
subquery1_1
main_query


,NAME_NODE,LABEL_NODE,WHERE,FUNCTION,ON
0,subquery1_5,query1@subquery1_5,orders.status = 'Completed',subquery,[]
1,orders,orders,None,table,
2,subquery1_4,query1@subquery1_4,None,subquery,[]
3,reviews,reviews,None,table,
4,subquery1_3,query1@subquery1_3,None,subquery,"[order_items.product_id = products.product_id,..."
5,order_items,order_items,None,table,
6,products,products,None,table,
7,subquery1_2,query1@subquery1_2,None,subquery,[]
8,payments,payments,None,table,
9,subquery1_1,query1@subquery1_1,orders.order_date >= DATE_ADD(CURRENT_TIMESTAM...,subquery,[]


In [27]:
nodes_dfs = []

for i, file in enumerate(os.listdir("gpt-sql-queries-extracted3")):
    filename = file.split('.')[0]

    files.append(file.split('.')[0])
    with open(f'gpt-sql-queries-extracted3/{file}', 'r') as file:
        data = json.load(file)

    queries.append(data)

    query_subqueries = dict(reversed(list(queries[i]['subquery_dictionary'].items())))
    query_subqueries['main_query'] = queries[i]['modified_SQL_query']

    nodes = []
    for name_query in query_subqueries:
        print(name_query)

        #query = query_subqueries[name_query].strip("()") # strip paranthesis from subqueries

        query = query_subqueries[name_query]
        if query.startswith("("): # remove open and closing paranthesis from subqueries
            query = query.strip("()")
        else:
            pass
        
        ast = parse_query(query) # get parsed tree
        
        tree = replace_aliases(query) # get transformed tree without table aliases
        
        # parse selects before getting statements
        select_statement_big = tree.find_all(exp.Select)

        source_tables = []
        # for every select statement in query, extract the source tables, where expressions and on conditions
        for select in list(select_statement_big):
            
            source_table, where_exp = get_statements(select) 

            on_condition = on_statement(select)

            source_tables += source_table

        if 'subquery' in name_query: # if the query is a subquery then the name is the dict key, else the name is the target table

            target_node = name_query
            nodes.append({'NAME_NODE': target_node, 'LABEL_NODE': f'{filename}@{target_node}', 'WHERE': where_exp, 'FUNCTION': 'subquery', 'ON': on_condition})
        else:
            try:
                target_node = list(ast.find_all(exp.Create))[0].this.this.this
                nodes.append({'NAME_NODE': target_node,'LABEL_NODE': target_node, 'WHERE': where_exp, 'FUNCTION': 'target', 'ON': on_condition})
            except IndexError:
                target_node = list(ast.find_all(exp.Insert))[0].this.this
                print(target_node)
                nodes.append({'NAME_NODE': target_node,'LABEL_NODE': target_node, 'WHERE': where_exp, 'FUNCTION': 'target', 'ON': on_condition})


        for table in source_tables:
            if table not in [node['NAME_NODE'] for node in nodes]:
                if 'subquery' in table: 
                    nodes.append({'NAME_NODE': table,'LABEL_NODE': f'{filename}@{table}', 'WHERE': None, 'FUNCTION': 'subquery', 'ON': on_condition})
                else:
                    nodes.append({'NAME_NODE': table,'LABEL_NODE': table, 'WHERE': None, 'FUNCTION': 'table', 'ON': None})
                    
    nodes = pd.DataFrame(nodes)
    nodes['COLOR'] = 'grey'

    nodes_dfs.append(pd.DataFrame(nodes))




subquery1_5
subquery1_4
subquery1_3
subquery1_2
subquery1_1


main_query
subquery1_2
subquery1_1
main_query
subquery2_1
subquery1_4
subquery1_3
subquery1_2
subquery1_1
main_query
subquery1_3
subquery1_2
subquery1_1
main_query
subquery1_4
subquery1_3
subquery1_2
subquery1_1
main_query


In [30]:
nodes = pd.concat(nodes_dfs).reset_index(drop=True)
nodes['ON'] = nodes['ON'].apply(lambda x: None if x == [] else x)

nodes = nodes.drop_duplicates(subset=['NAME_NODE', 'LABEL_NODE', 'WHERE', 'FUNCTION']).reset_index(drop=True)

nodes['ID'] = nodes.index

nodes.to_csv('output-data/nodes.csv')

nodes


,NAME_NODE,LABEL_NODE,WHERE,FUNCTION,ON,COLOR,ID
0,subquery1_5,json_data0@subquery1_5,orders.status = 'Completed',subquery,None,grey,0
1,orders,orders,None,table,None,grey,1
2,subquery1_4,json_data0@subquery1_4,None,subquery,None,grey,2
3,reviews,reviews,None,table,None,grey,3
4,subquery1_3,json_data0@subquery1_3,None,subquery,"[order_items.product_id = products.product_id,...",grey,4
5,order_items,order_items,None,table,None,grey,5
6,products,products,None,table,None,grey,6
7,subquery1_2,json_data0@subquery1_2,None,subquery,None,grey,7
8,payments,payments,None,table,None,grey,8
9,subquery1_1,json_data0@subquery1_1,orders.order_date >= DATE_ADD(CURRENT_TIMESTAM...,subquery,None,grey,9


In [ ]:
['LABEL_NODE','ID','FUNCTION','JOIN_ARGU','NAME_NODE','FILTER','COLOR']
#1,B_BillingHdr@BillingDocHdr_P,1,ProjectionView,,BillingDocHdr_P,,black


# distinguish row level filter from join arguments (on and wheres)

['LABEL_NODE', 'ID', 'FUNCTION', 'JOIN_ARGU', 'NAME_NODE', 'FILTER', 'COLOR']